# Verifying SRD Requirements for DESI LRGs

Using the Cascades Internal Data release and updated for the 1% Survey selection

In [ ]:
from importlib import reload
from pathlib import Path

import numpy as np
from astropy.table import Table, vstack, hstack, unique,join
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

from desitarget.sv3.sv3_targetmask import desi_mask
import utils
reload(utils)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

Effective areas for tiles in Cascades

In [ ]:
#in square degrees
#obtained from .dat files `/global/cfs/cdirs/desi/survey/catalogs/SV1/LSS/LSScats/cascades_1`
#First row of tiles are from Blanc era the next rows are from cascades
areas = {80605:1.3598, 80607:1.13, 80609:1.3628, 80620:1.1664, 80622:1.1108,
         80674:1.0142, 80676:1.0222, 80678:0.9804, 80680:1.0768,80708:0.981,
         80682:1.1446, 80684:0.9574, 80688:1.145,  80690:1.0748,80670:1.2246,  
         80692:0.9072, 80694:0.8778, 80700:1.1644, 80686:1.1666, 80712:0.97,
         
        }

areas = pd.DataFrame(areas.items(), columns=["TILEID", "AREA"]).sort_values(by="TILEID")
areas = areas.set_index("TILEID", drop=False)

# Load necessary data

In [ ]:
# read file into an astropy table and move it to pandas
# pandas is objectively BETTER as can be seen below
z_path = Path("/global/cfs/cdirs/desi/survey/catalogs/SV1/redshift_comps/cascades/3.1/All/alltiles_Allzinfo_wrz.fits")
z_cat = Table.read(z_path)
names = [name for name in z_cat.colnames if len(z_cat[name].shape) <= 1]
z_cat = z_cat[names].to_pandas()
z_cat = z_cat[(z_cat["FIBERSTATUS"]==0)] #select only good fibers
z_cat = z_cat[z_cat["TARGETS"] == b"QSO+LRG"] # Use only QSO+LRG Tiles
z_cat = z_cat[z_cat["TILEID"].isin(areas["TILEID"].values)] # Use tiles from the above dictionary

In [ ]:
#Ashleys catalog does not have MW extinctions
my_path = Path("/global/cscratch1/sd/bid13/LRG")

lrg_cat = (Table.read(my_path / ("LRG_SV3.fits")).to_pandas()).convert_dtypes(convert_integer=False)
z_cat = z_cat.merge(lrg_cat, on="TARGETID", how="inner", validate="many_to_one")

z_cat_single = z_cat[(z_cat["EFFTIME_DARK"]>800) & (z_cat["EFFTIME_DARK"]<1200)] # Use data with effective exposure time of about 1000s
z_cat_asgn = z_cat[z_cat["subset"]==b"deep"]
z_cat_deep = z_cat_asgn[z_cat_asgn["EFFTIME_DARK"]>3000] # Deep coadds to get the redshift range
# There are duplicates in z_cat_deep since some targets were assigned in two tiles. 

del lrg_cat

### Assignment Densities for each tile
The effective areas calculated for the tiles used here are not very trustworthy. So, all the calculations will be done in percentages. This section lists the assignment densities using the effective areas.

In [ ]:
lrg_mask = (z_cat_asgn["SV3_DESI_TARGET"].array & desi_mask.mask("LRG"))>0
lrg_low_mask = (z_cat_asgn["SV3_DESI_TARGET"] & desi_mask.mask("LRG_LOWDENS"))>0

In [ ]:
areas["LRG_count"] = z_cat_asgn[lrg_mask].groupby("TILEID", sort=True).count()["TARGETID"].values
areas["LRG_dens"] = areas["LRG_count"]/areas["AREA"]
areas["LRG_LOWDENS_count"] = z_cat_asgn[lrg_low_mask].groupby("TILEID", sort=True).count()["TARGETID"].values
areas["LRG_LOWDENS_dens"] = areas["LRG_LOWDENS_count"]/areas["AREA"]

In [ ]:
areas

### **L2.2.1** The average density of successfully observed, galaxy clustering science-quality LRGs with redshift $0.3 < z < 1.0$ shall be at least $300$ deg $^{-2}$

### Fraction of objects in the correct redshift ranges as calculated from the deep coadds

**Fraction of bad redshifts in deep = star OR DELTACHI2<15 OR redshift>1.4 / All measurements** 

**Fraction in correct redshift range = correct redshift range / all good measurements** (definition of good in the above line)

In [ ]:
lrg_mask = (z_cat_deep["SV3_DESI_TARGET"].array & desi_mask.mask("LRG"))>0
lrg_low_mask = (z_cat_deep["SV3_DESI_TARGET"] & desi_mask.mask("LRG_LOWDENS"))>0

In [ ]:
star_mask = (z_cat_deep["SPECTYPE"]!=b"STAR")
deltachi2_mask = (z_cat_deep["DELTACHI2"]>15)
z_mask = (z_cat_deep["Z"]<=1.4)

z_mask_narrow = (z_cat_deep["Z"]>=0.3) &  (z_cat_deep["Z"]<=1.0)
z_mask_wide = (z_cat_deep["Z"]>=0.3) &  (z_cat_deep["Z"]<=1.4)

In [ ]:
grouped_all = z_cat_deep.groupby(by="TILEID", sort=True)

grouped_bad = z_cat_deep[ ( (~star_mask) | (~deltachi2_mask) | (~z_mask)) & lrg_mask].groupby(by="TILEID", sort=True)
print(f"Bad redshifts % in LRG (deep): { 100* grouped_bad.count().Z.sum()/grouped_all.count().Z.sum():.3f}")

grouped_bad = z_cat_deep[ ( (~star_mask) | (~deltachi2_mask) | (~z_mask)) & lrg_low_mask].groupby(by="TILEID", sort=True)
print(f"Bad redshifts % in LRG_LOWDENS (deep): {100* grouped_bad.count().Z.sum()/grouped_all.count().Z.sum():.3f}")

**Fraction of good objects in redshift range 0.3-1.0**

In [ ]:
grouped = z_cat_deep[star_mask & deltachi2_mask & z_mask & z_mask_narrow & lrg_mask].groupby(by="TILEID", sort=True)
grouped_all = z_cat_deep[star_mask & deltachi2_mask & z_mask & lrg_mask].groupby(by="TILEID", sort=True)
print(f"Correct range among good redshifts % LRG: {100 * grouped.count().Z.sum()/grouped_all.count().Z.sum():.3f}")

grouped = z_cat_deep[star_mask & deltachi2_mask & z_mask & z_mask_narrow & lrg_low_mask].groupby(by="TILEID", sort=True)
grouped_all = z_cat_deep[star_mask & deltachi2_mask & z_mask & lrg_low_mask].groupby(by="TILEID", sort=True)
print(f"Correct range among good redshifts % LRG_LOWDENS: {100 * grouped.count().Z.sum()/grouped_all.count().Z.sum():.3f}")

grouped = z_cat_deep[star_mask & deltachi2_mask & z_mask & z_mask_narrow & lrg_mask].groupby(by="TILEID", sort=True)
grouped_all = z_cat_deep[lrg_mask].groupby(by="TILEID", sort=True)
print(f"Correct range among all redshifts % LRG: {100 * grouped.count().Z.sum()/grouped_all.count().Z.sum():.3f}")

grouped = z_cat_deep[star_mask & deltachi2_mask & z_mask & z_mask_narrow & lrg_low_mask].groupby(by="TILEID", sort=True)
grouped_all = z_cat_deep[lrg_low_mask].groupby(by="TILEID", sort=True)
print(f"Correct range among all redshifts % LRG_LOWDENS: {100 * grouped.count().Z.sum()/grouped_all.count().Z.sum():.3f}")

**Fraction of good objects in redshift range 0.3-1.5**

In [ ]:
grouped = z_cat_deep[star_mask & deltachi2_mask & z_mask & z_mask_wide & lrg_mask].groupby(by="TILEID", sort=True)
grouped_all = z_cat_deep[star_mask & deltachi2_mask & z_mask & lrg_mask].groupby(by="TILEID", sort=True)
print(f"Correct range among good redshifts % LRG: {100 * grouped.count().Z.sum()/grouped_all.count().Z.sum():.3f}")

grouped = z_cat_deep[star_mask & deltachi2_mask & z_mask & z_mask_wide & lrg_low_mask].groupby(by="TILEID", sort=True)
grouped_all = z_cat_deep[star_mask & deltachi2_mask & z_mask & lrg_low_mask].groupby(by="TILEID", sort=True)
print(f"Correct range among good redshifts % LRG_LOWDENS: {100 * grouped.count().Z.sum()/grouped_all.count().Z.sum():.3f}")

grouped = z_cat_deep[star_mask & deltachi2_mask & z_mask & z_mask_wide & lrg_mask].groupby(by="TILEID", sort=True)
grouped_all = z_cat_deep[lrg_mask].groupby(by="TILEID", sort=True)
print(f"Correct range among all redshifts % LRG: {100 * grouped.count().Z.sum()/grouped_all.count().Z.sum():.3f}")

grouped = z_cat_deep[star_mask & deltachi2_mask & z_mask & z_mask_wide & lrg_low_mask].groupby(by="TILEID", sort=True)
grouped_all = z_cat_deep[lrg_low_mask].groupby(by="TILEID", sort=True)
print(f"Correct range among all redshifts % LRG_LOWDENS: {100 * grouped.count().Z.sum()/grouped_all.count().Z.sum():.3f}")

### Fraction of successfull measurements using 1x depth

A successful measurement is defined as:
- `DELTACHI2` > 15 in deep coadd
- `ZWARN` = 0 in every single visit
- repeatable in the single visits
- z < 1.4 in deep coadd

A repeatable measurement is defined as one for which each single exposure measurement satisfies $\Delta z < 0.0033 (1+z)$ wrt deep coadds. We will only consider measurements for which 1x depth repeat observations are available.

In [ ]:
# Select only exposures for which repeatability analysis is possible
z_cat_single["mask"] = np.zeros(len(z_cat_single), dtype="bool")
for tile in areas["TILEID"].values:
    temp_mask = z_cat_single[z_cat_single["TILEID"]==tile].duplicated(subset="TARGETID", keep=False)
    z_cat_single.loc[z_cat_single["TILEID"]==tile, "mask"]=temp_mask.values
z_cat_single = z_cat_single[z_cat_single["mask"].values]   

In [ ]:
# Restrict deep coadds to the ones having multiple observations
# Select only exposures for which repeatability analysis is possible
z_cat_deep["mask"] = np.zeros(len(z_cat_deep), dtype="bool")
for tile in areas["TILEID"].values:
    temp_mask = z_cat_deep[z_cat_deep["TILEID"]==tile]["TARGETID"].isin(np.unique(z_cat_single[z_cat_single["TILEID"]==tile]["TARGETID"]))
    z_cat_deep.loc[z_cat_deep["TILEID"]==tile, "mask"]=temp_mask.values
z_cat_deep = z_cat_deep[z_cat_deep["mask"].values]   
z_cat_single = z_cat_single[z_cat_single["TARGETID"].isin(np.unique(z_cat_deep["TARGETID"]))]

# 80694 has 7 and 80678 has 20 usable observations. Maybe remove them in future?

In [ ]:
# Find successful measurements
z_cat_deep["success"] = (z_cat_deep["DELTACHI2"]>15).values
z_cat_deep["success"] &= (z_cat_deep["Z"]<1.4).values
deep_tile_group = z_cat_deep.groupby("TILEID")
single_tile_group = z_cat_single.groupby("TILEID")
for tile, deep_tile_subset in deep_tile_group:
    deep_target_group = deep_tile_subset.groupby("TARGETID")
    single_target_group = single_tile_group.get_group(tile).groupby("TARGETID")
#     print("")
#     print(tile, end="\r")
#     print("")
    for target, single_target_subset in single_target_group:
        z_warn_mask = np.all(single_target_subset["ZWARN"]==0)
        z_true = deep_target_group.get_group(target)["Z"]
        z_errors = np.abs(single_target_subset["Z"] -  z_true)/(1 + z_true)
        z_repeat_mask = np.all(z_errors<0.0033333)
        deep_target_group.get_group(target)["success"] &= (z_warn_mask&z_repeat_mask)

In [ ]:
lrg_mask = (z_cat_deep["SV3_DESI_TARGET"].array & desi_mask.mask("LRG"))>0
lrg_low_mask = (z_cat_deep["SV3_DESI_TARGET"] & desi_mask.mask("LRG_LOWDENS"))>0

In [ ]:
print(f"% of successfull measurements at 1x depth (relative to all measurements)")
print(f"LRG: {(100*z_cat_deep.success[lrg_mask]).sum()/lrg_mask.sum():.3f} ")
print(f"LRG_LOWDENS: {100*z_cat_deep.success[lrg_low_mask].sum()/lrg_low_mask.sum():.3f} ")

In [ ]:
print(f"% of successfull measurements (relative to all measurements) at 1x depth \nWithin redshifts 0.3 to 1 and not star")
z_mask = ((z_cat_deep.Z>=0.3)&(z_cat_deep.Z<=1))
star_mask = z_cat_deep.SPECTYPE != b"STAR"
print(f"LRG: {(100*z_cat_deep.success[lrg_mask&z_mask&star_mask]).sum()/lrg_mask.sum():.3f} ")
print(f"LRG_LOWDENS: {100*z_cat_deep.success[lrg_low_mask&z_mask&star_mask].sum()/lrg_low_mask.sum():.3f} ")

In [ ]:
print(f"% of successfull measurements (relative to all measurements) at 1x depth \nWithin redshifts 0.3 to 1.5 and not star")
z_mask = ((z_cat_deep.Z>=0.3)&(z_cat_deep.Z<=1.5))
star_mask = z_cat_deep.SPECTYPE != b"STAR"
print(f"LRG: {(100*z_cat_deep.success[lrg_mask&z_mask&star_mask]).sum()/lrg_mask.sum():.3f} ")
print(f"LRG_LOWDENS: {100*z_cat_deep.success[lrg_low_mask&z_mask&star_mask].sum()/lrg_low_mask.sum():.3f} ")

In [ ]:
print(f"% Redshift range of 0.3 to 1 and not star at 1x depth relative to successful measurements")
z_mask = ((z_cat_deep.Z>=0.3)&(z_cat_deep.Z<=1))
star_mask = z_cat_deep.SPECTYPE != b"STAR"
print(f"LRG: {(100*z_cat_deep.success[lrg_mask&z_mask&star_mask]).sum()/z_cat_deep.success[lrg_mask].sum():.3f} ")
print(f"LRG_LOWDENS: {100*z_cat_deep.success[lrg_low_mask&z_mask&star_mask].sum()/z_cat_deep.success[lrg_low_mask].sum():.3f} ")

In [ ]:
print(f"% Redshift range of 0.3 to 1.4 and not star at 1x depth relative to successful measurements")
z_mask = ((z_cat_deep.Z>=0.3)&(z_cat_deep.Z<=1.4))
star_mask = z_cat_deep.SPECTYPE != b"STAR"
print(f"LRG: {(100*z_cat_deep.success[lrg_mask&z_mask&star_mask]).sum()/z_cat_deep.success[lrg_mask].sum():.3f} ")
print(f"LRG_LOWDENS: {100*z_cat_deep.success[lrg_low_mask&z_mask&star_mask].sum()/z_cat_deep.success[lrg_low_mask].sum():.3f} ")

In [ ]:
num_bins =10
z_bins = np.linspace(0.3,1.4, num_bins+1)
z_success = []
zfiber_bins = np.linspace(z_cat_deep.zfibermag.min(),z_cat_deep.zfibermag.max(), num_bins+1)
zfiber_success = []
fig , ax = plt.subplots(1,2, figsize=(15,5))
ax = np.ravel(ax)
for i in range(num_bins):
    temp_df = z_cat_deep[(z_cat_deep.Z>=z_bins[i]) & (z_cat_deep.Z<z_bins[i+1])]
    z_success.append(100* temp_df.success.sum()/len(temp_df))
    temp_df = z_cat_deep[(z_cat_deep.zfibermag>=zfiber_bins[i]) & (z_cat_deep.zfibermag<zfiber_bins[i+1])]
    zfiber_success.append(100* temp_df.success.sum()/len(temp_df))
ax[0].set_xlabel("Spectroscopic Redshift", size=20)
ax[0].set_ylabel("Percentage of Success", size=20)
ax[0].set_ylim(98, 101)
utils.better_step(z_bins, z_success, ax[0])
ax[1].set_xlabel("z-fiber mag", size=20)
ax[1].set_ylabel("Percentage of Success", size=20)
ax[1].set_ylim(98, 101)
utils.better_step(zfiber_bins, zfiber_success, ax[1])

### **L2.2.2** The random error on individual LRG redshifts in a ∼Gaussian core shall be less than $\sigma_{z} = 0.0005(1 + z)$ (equivalent to 150 km/s rms).


Here we consider each 1x depth measurement with multiple observations. We also restrict to measurements with:
- `ZWARN` = 0 
- `SPECTYPE`!= "STAR"
- `Z`<1.4
- `DELTACHI2<15`

For them we calculate every possible pair of $\Delta z/(1+z)$ and measure their spread and outlier rate.

In [ ]:
z_cat_single = z_cat[(z_cat["EFFTIME_DARK"]>800) & (z_cat["EFFTIME_DARK"]<1200)] # Use data with effective exposure time of about 1000s

# Select only exposures for which repeatability analysis is possible
z_cat_single["mask"] = np.zeros(len(z_cat_single), dtype="bool")
for tile in areas["TILEID"].values:
    temp_mask = z_cat_single[z_cat_single["TILEID"]==tile].duplicated(subset="TARGETID", keep=False)
    z_cat_single.loc[z_cat_single["TILEID"]==tile, "mask"]=temp_mask.values
z_cat_single = z_cat_single[z_cat_single["mask"].values]   

In [ ]:
z_cat_single = z_cat_single[z_cat_single["ZWARN"]==0]
z_cat_single = z_cat_single[z_cat_single["SPECTYPE"]!=b"STAR"]
z_cat_single = z_cat_single[z_cat_single["DELTACHI2"]>15]
z_cat_single = z_cat_single[z_cat_single["Z"]<1.4]

In [ ]:
lrg_mask = (z_cat_single["SV3_DESI_TARGET"].array & desi_mask.mask("LRG"))>0
lrg_low_mask = (z_cat_single["SV3_DESI_TARGET"] & desi_mask.mask("LRG_LOWDENS"))>0

In [ ]:
all_errors_lrg=np.array([])
z_single_grouped_lrg = (z_cat_single[lrg_mask]).groupby("TARGETID")
for target, group in z_single_grouped_lrg:
    z = group["Z"].values
    zerrors = np.abs(z[None, :] - z[:, None])/(1+z[None, :]) #find all possible errors
    zerrors = zerrors[~np.eye(len(z),dtype="bool")] # Select
    all_errors_lrg = np.concatenate([all_errors_lrg, zerrors])

all_errors_lrg_low=np.array([])
z_single_grouped_lrg_low = (z_cat_single[lrg_low_mask]).groupby("TARGETID")
for target, group in z_single_grouped_lrg_low:
    z = group["Z"].values
    zerrors = np.abs(z[None, :] - z[:, None])/(1+z[None, :]) #find all possible errors
    zerrors = zerrors[~np.eye(len(z),dtype="bool")]
    all_errors_lrg_low = np.concatenate([all_errors_lrg_low, zerrors])

In [ ]:
print(f"LRG σ_z = {stats.median_abs_deviation(all_errors_lrg,scale='normal')/np.sqrt(2):.5f}(1+z)")
print(f"LRG_LOWDENS σ_z = {stats.median_abs_deviation(all_errors_lrg_low,scale='normal')/np.sqrt(2):.5f}(1+z)")

In [ ]:
num_bins =10
z_bins = np.linspace(0.3,1.4, num_bins+1)
z_error_lrg = []
z_error_lrg_low = []
zfiber_bins = np.linspace(z_cat_single.zfibermag.min(),z_cat_single.zfibermag.max(), num_bins+1)
zfiber_error_lrg = []
zfiber_error_lrg_low = []
fig , ax = plt.subplots(1,2, figsize=(15,5))
ax = np.ravel(ax)
for i in range(num_bins):
    all_errors_lrg_=np.array([])
    # Maybe bin using the deep redshifts?
    z_single_grouped_lrg = (z_cat_single[lrg_mask & (z_cat_single.Z>=z_bins[i]) & (z_cat_single.Z<z_bins[i+1])]).groupby("TARGETID")
    for target, group in z_single_grouped_lrg:
        z = group["Z"].values
        zerrors = np.abs(z[None, :] - z[:, None])/(1+z[None, :]) #find all possible errors
        zerrors = zerrors[~np.eye(len(z),dtype="bool")] # Select
        all_errors_lrg_ = np.concatenate([all_errors_lrg_, zerrors])
    z_error_lrg.append(stats.median_abs_deviation(all_errors_lrg_,scale='normal')/np.sqrt(2))
    
    all_errors_lrg_=np.array([])
    # Maybe bin using the deep redshifts?
    z_single_grouped_lrg = (z_cat_single[lrg_mask & (z_cat_single.zfibermag>=zfiber_bins[i]) & (z_cat_single.zfibermag<zfiber_bins[i+1])]).groupby("TARGETID")
    for target, group in z_single_grouped_lrg:
        z = group["Z"].values
        zerrors = np.abs(z[None, :] - z[:, None])/(1+z[None, :]) #find all possible errors
        zerrors = zerrors[~np.eye(len(z),dtype="bool")] # Select
        all_errors_lrg_ = np.concatenate([all_errors_lrg_, zerrors])
    zfiber_error_lrg.append(stats.median_abs_deviation(all_errors_lrg_,scale='normal')/np.sqrt(2))
    
    all_errors_lrg_=np.array([])
    z_single_grouped_lrg = (z_cat_single[lrg_low_mask & (z_cat_single.Z>=z_bins[i]) & (z_cat_single.Z<z_bins[i+1])]).groupby("TARGETID")
    for target, group in z_single_grouped_lrg:
        z = group["Z"].values
        zerrors = np.abs(z[None, :] - z[:, None])/(1+z[None, :]) #find all possible errors
        zerrors = zerrors[~np.eye(len(z),dtype="bool")] # Select
        all_errors_lrg_ = np.concatenate([all_errors_lrg_, zerrors])
    z_error_lrg_low.append(stats.median_abs_deviation(all_errors_lrg_,scale='normal')/np.sqrt(2))
    
    all_errors_lrg_=np.array([])
    z_single_grouped_lrg = (z_cat_single[lrg_low_mask & (z_cat_single.zfibermag>=zfiber_bins[i]) & (z_cat_single.zfibermag<zfiber_bins[i+1])]).groupby("TARGETID")
    for target, group in z_single_grouped_lrg:
        z = group["Z"].values
        zerrors = np.abs(z[None, :] - z[:, None])/(1+z[None, :]) #find all possible errors
        zerrors = zerrors[~np.eye(len(z),dtype="bool")] # Select
        all_errors_lrg_ = np.concatenate([all_errors_lrg_, zerrors])
    zfiber_error_lrg_low.append(stats.median_abs_deviation(all_errors_lrg_,scale='normal')/np.sqrt(2))
    
    
ax[0].set_xlabel("Spectroscopic Redshift", size=20)
ax[0].set_ylabel("$\sigma_{z}$", size=20)
ax[0].axhline(0.0005, ls="--", c= "k", label="Requirement")
utils.better_step(z_bins, z_error_lrg, ax[0], label="LRG")
utils.better_step(z_bins, z_error_lrg_low, ax[0], label="LRG_LOWDENS", ls="--")
# ax[0].set_ylim(0, 0.00052)
ax[1].set_xlabel("z-fiber mag", size=20)
ax[1].set_ylabel("$\sigma_{z}$", size=20)
ax[1].axhline(0.0005, ls="--", c= "k", label="Requirement")
# ax[1].set_ylim(0, 0.00052)
utils.better_step(zfiber_bins, zfiber_error_lrg, ax[1], label="LRG")
utils.better_step(zfiber_bins, zfiber_error_lrg_low, ax[1], label="LRG_LOWDENS", ls="--")
plt.legend()

### **L2.2.3** Systematic inaccuracy in the mean LRG redshift shall be less than $\Delta z = 0.0002(1+ z)$ (equivalent to 60 km/s).

See separate notebook which uses stacking to check this.

### **L2.2.4** LRG redshift catastrophic failures exceeding 1000 km/s shall be < 5%.

In [ ]:
print("Catastrophic Outlier %")
frac = (all_errors_lrg>0.00333).sum()*100/len(all_errors_lrg)
print(f"LRG: {frac/2:.3f}-{frac:.3f}%")
frac = (all_errors_lrg_low>0.00333).sum()*100/len(all_errors_lrg_low)
print(f"LRG_LOWDENS: {frac/2:.3f}-{frac:.3f}%")

In [ ]:
# Plot the above as a function of redshift and zfiber?

### **L2.2.5** The LRG redshift completeness shall be >95% for each pointing averaged over all targets that receive fibers.

See section 2.2.1 for the fractions of successfull measurements/total measurements. NaN values indicate no elligible objects were found.

In [ ]:
lrg_mask = (z_cat_deep["SV3_DESI_TARGET"].array & desi_mask.mask("LRG"))>0
lrg_low_mask = (z_cat_deep["SV3_DESI_TARGET"] & desi_mask.mask("LRG_LOWDENS"))>0

group_lrg = (z_cat_deep[lrg_mask]).groupby("TILEID")
group_lrg_low = (z_cat_deep[lrg_low_mask]).groupby("TILEID")

completeness = pd.DataFrame({ "LRG":(100*group_lrg["success"].sum()/group_lrg["success"].count()),
                             "LRG_count": group_lrg["success"].count(),
                             "LRG_LOWDENS":(100*group_lrg_low["success"].sum()/group_lrg_low["success"].count()),
                             "LRG_LOWDENS_count": group_lrg_low["success"].count(),
                             
}, index=areas["TILEID"])
completeness

### **L2.8.1** The LRG target density shall be 350 per deg2, with at least 300 per deg2 successfully measured (§L2.2.1). With a fiber assignment efficiency of 99% for LRGs, this implies a combined target selection and redshift measurement efficiency of 87%. The magnitude limit for this sample is z$_{\mathrm{AB}} < 20.56$.


See section 2.2.1 for the fractions of successfull measurements in the given redshift range.